In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier



In [2]:
import tensorflow as tf
print(tf.__version__)

2.10.1


In [3]:
import keras
keras.__version__

'2.10.0'

In [4]:
train_df=pd.read_csv('artifacts/train.csv')
test_df=pd.read_csv('artifacts/test.csv')
val_df=pd.read_csv('artifacts/validation.csv')

In [5]:
train_df

,label_1,label_X,label_2,rounds_left,win_share_latest_5_games_overall_home_team,draw_share_latest_5_games_overall_home_team,loss_share_latest_5_games_overall_home_team,avg_goals_scored_latest_5_games_overall_home_team,avg_goals_conceded_latest_5_games_overall_home_team,win_share_latest_5_games_overall_away_team,...,player_diff_max_f_rolling_mean_6_impact_game_threat,player_diff_max_f_rolling_mean_6_impact_game_transfers_balance,player_diff_max_f_rolling_mean_6_impact_game_hourly_rate_goals_scored,player_diff_max_f_rolling_mean_6_impact_game_hourly_rate_assists,player_diff_max_f_rolling_max_6_impact_game_influence,player_diff_max_f_rolling_max_6_impact_game_creativity,player_diff_max_f_rolling_max_6_impact_game_threat,player_diff_max_f_rolling_max_6_impact_game_transfers_balance,player_diff_max_f_rolling_max_6_impact_game_hourly_rate_goals_scored,player_diff_max_f_rolling_max_6_impact_game_hourly_rate_assists
0,0,1,0,22,0.2,0.0,0.8,0.8,1.8,0.4,...,0.424145,538.246856,0.010437,0.010406,1.393622,4.391184,-0.041120,1935.761004,0.001829,6.243496e-02
1,0,0,1,8,0.4,0.6,0.0,0.8,0.2,0.2,...,-0.237234,-962.402354,0.000010,0.000000,-0.069866,0.339786,-0.085648,3066.740891,0.000062,0.000000e+00
2,0,0,1,23,0.0,0.2,0.8,0.8,2.0,0.6,...,-1.791782,-4634.318948,-0.010132,-0.019630,0.838057,0.401216,0.547112,-17562.788734,0.000000,3.312291e-03
3,0,0,1,14,0.2,0.4,0.4,1.2,1.8,1.0,...,0.537508,7251.600616,0.000041,-0.010111,1.154472,0.065181,2.738412,71884.883721,0.000123,6.938894e-18
4,1,0,0,-5,0.0,0.2,0.8,0.4,1.8,0.0,...,0.281974,1.130054,0.010132,0.010132,4.569604,0.110660,0.049238,-1846.785306,0.060790,6.079027e-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
642,1,0,0,-7,0.6,0.0,0.4,2.0,1.8,0.0,...,1.580998,31.626353,0.030405,0.010101,2.344523,-0.352484,5.819689,-2433.729582,0.060852,6.060606e-02
643,0,0,1,24,0.2,0.4,0.4,1.4,1.2,0.6,...,-0.871409,175.083477,0.010372,0.000000,2.950207,1.789588,-1.550152,462.753036,0.060790,0.000000e+00
644,0,0,1,23,0.4,0.0,0.6,1.6,1.8,0.6,...,-0.964750,-1304.099292,0.020233,-0.010173,-0.046227,-0.614925,-4.636364,-5387.099291,0.000123,-6.103764e-02
645,1,0,0,2,0.4,0.2,0.4,1.2,0.8,0.0,...,0.568503,1406.894837,-0.010050,-0.010132,-3.299906,-1.313070,1.581015,-3054.701851,-0.060300,-6.079027e-02


In [6]:
X_train = pd.concat([train_df, test_df]).drop(columns=['label_1', 'label_X', 'label_2'], axis=1).reset_index(drop=True)
y_train = pd.concat([train_df, test_df])[['label_1', 'label_X', 'label_2']].reset_index(drop=True)

X_val = val_df.drop(columns=['label_1', 'label_X', 'label_2'], axis=1).reset_index(drop=True)
y_val = val_df[['label_1', 'label_X', 'label_2']].reset_index(drop=True)

In [10]:
# Define your deep learning model with increased complexity
def create_model(hidden_units=[128, 64, 32], optimizer='adam'):
    model = Sequential()
    model.add(Dense(hidden_units[0], input_shape=(X_train.shape[1],), activation='relu'))
    for units in hidden_units[1:]:
        model.add(Dense(units, activation='relu'))
    model.add(Dense(3, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

# Create KerasClassifier for GridSearchCV
model = KerasClassifier(build_fn=create_model, epochs=10, batch_size=32)

# Define the hyperparameters to tune
parameters = {
    'optimizer': ['adam', 'sgd'],
    'hidden_units': [(128,), (64, 32), (32, 16, 8)]
}

# Create the GridSearchCV object
grid_search = GridSearchCV(estimator=model, param_grid=parameters, cv=3)

# Perform grid search on your training data
grid_search.fit(X_train, y_train)

# Print the best hyperparameters and the corresponding accuracy
print("Best Hyperparameters: ", grid_search.best_params_)
print("Best Accuracy: ", grid_search.best_score_)


C:\Users\Marcus Bergdahl\AppData\Local\Temp\ipykernel_17224\2826881907.py:12: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(build_fn=create_model, epochs=10, batch_size=32)


Epoch 1/10
18/18 [==============================] - 1s 2ms/step - loss: 7609.8633 - accuracy: 0.3704
Epoch 2/10
18/18 [==============================] - 0s 3ms/step - loss: 2754.2065 - accuracy: 0.4348
Epoch 3/10
18/18 [==============================] - 0s 2ms/step - loss: 1883.4785 - accuracy: 0.4435
Epoch 4/10
18/18 [==============================] - 0s 2ms/step - loss: 1527.4489 - accuracy: 0.4417
Epoch 5/10
18/18 [==============================] - 0s 2ms/step - loss: 1206.9678 - accuracy: 0.4730
Epoch 6/10
18/18 [==============================] - 0s 2ms/step - loss: 1104.4246 - accuracy: 0.4870
Epoch 7/10
18/18 [==============================] - 0s 2ms/step - loss: 980.2961 - accuracy: 0.5235
Epoch 8/10
18/18 [==============================] - 0s 2ms/step - loss: 916.8569 - accuracy: 0.5165
Epoch 9/10
18/18 [==============================] - 0s 1ms/step - loss: 902.4522 - accuracy: 0.5357
Epoch 10/10
9/9 [==============================] - 0s 2ms/step - loss: 2499.2644 - accuracy: 0

In [12]:
# Evaluate the model on the validation set
best_model = grid_search.best_estimator_
result = best_model.predict_proba(X_val)
print("Validation Predictions:")
print(result)

27/27 [==============================] - 0s 1ms/step
Validation Predictions:
[[nan nan nan]
 [nan nan nan]
 [nan nan nan]
 ...
 [nan nan nan]
 [nan nan nan]
 [nan nan nan]]
